In [1]:
## pip install moviepy==1.0.0

import os
import librosa
import pandas as pd
from moviepy.editor import *
from tqdm import tqdm_notebook
import numpy as np
import re

In [2]:
#from utils import make_dir
def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

In [3]:
audio_label_clip_dir = os.path.join('.', 'data', 'audio_label_clip')
make_dir(audio_label_clip_dir)

In [4]:
def save(audios, label):
    data = pd.DataFrame({
        'audios':audios, 'label':label })
    data.to_csv(audio_label_clip_dir+'/audio_label_clip.csv', index=False, encoding='euc-kr')

In [5]:
## 레이블링 데이터 위치 : ./data/label
files = []
for file in os.listdir(os.path.join('.', 'data', 'label')):
    _, file_id = os.path.split(file)
    if file_id.endswith('txt'):
        name = file_id.split(".")[0]
        if(name not in files):
            files.append(name)

print(files)

['bGFkdfYPPAo_0', 'bGFkdfYPPAo_1', 'bGFkdfYPPAo_10', 'bGFkdfYPPAo_11', 'bGFkdfYPPAo_12', 'bGFkdfYPPAo_13', 'bGFkdfYPPAo_14', 'bGFkdfYPPAo_15', 'bGFkdfYPPAo_16', 'bGFkdfYPPAo_17', 'bGFkdfYPPAo_18', 'bGFkdfYPPAo_2', 'bGFkdfYPPAo_3', 'bGFkdfYPPAo_4', 'bGFkdfYPPAo_5', 'bGFkdfYPPAo_6', 'bGFkdfYPPAo_7', 'bGFkdfYPPAo_8', 'bGFkdfYPPAo_9']


In [6]:
audio = []
label = []
shi = 0
jon = 0
sha = 0
byoung = 0
background = 0
negative = 0
count = 0

for file in files:
    print("file name:" + file)
    text = './data/label/' + file + ".txt"
    aud = './data/label/' + file + ".wav"
    # 정규표현식 패턴
    pattern = r"^(.*?)_\d+$"
    # 정규표현식 매칭
    match = re.match(pattern, file)
    raw_aud = './data/audio/' + str(match.group(1))+ ".wav"

    print("read new file")
    data = pd.read_csv(text, sep="\t", names=['start', 'end', 'label'])
    print(data)
#     data.start.astype('float')
#     data.end.astype('float')
    for i in range(len(data)):
        lab = data.label[i]
        print(data.start[i], data.end[i], data.label[i])
        try:
            start = float(data.start[i])
            end = float(data.end[i])
        except ValueError:
            continue
        if start >= end:
            continue
        if np.array(data.label[i])==np.nan:
            continue
        full_audio = AudioFileClip(raw_aud)
        if(lab==10): #shi
            name = "shi_" + str(shi)
            shi += 1
        elif(lab==11): #jon 
            name = "jon_" + str(jon)
            jon += 1
        elif(lab==12): #sha
            name = "sha_" + str(sha)
            sha += 1
        elif(lab==13): #byoung
            name = "byoung_" + str(byoung)
            byoung += 1
        elif(lab==9): #background
            name = "background_" + str(background)
            background+= 1
        elif(lab==0): #negative
            name = "negative_" + str(negative)
            negative += 1
        tmp_audio = full_audio.subclip(start, end)
        name = os.path.join(audio_label_clip_dir,  name + ".wav")
        try:
            tmp_audio.write_audiofile(name)
        except OSError:
            print('oserror')
            continue
            
        audio.append(name + ".wav")
        label.append(int(lab))
        print('완료됬습니다.')
    count += 1
save(audio, label)

file name:bGFkdfYPPAo_0
read new file
   start  end  label
0  0.539  3.6     10
0.539 3.6 10
MoviePy - Writing audio in .\data\audio_label_clip\shi_0.wav
MoviePy - Done.
dd
file name:bGFkdfYPPAo_1
read new file
   start   end  label
0    3.6  4.62      0
3.6 4.62 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_0.wav

chunk:   0%|          | 0/32 [00:00<?, ?it/s, now=None]


MoviePy - Done.
dd
file name:bGFkdfYPPAo_10
read new file
    start    end  label
0  25.439  26.88      0
25.439 26.88 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_1.wav


MoviePy - Done.
dd
file name:bGFkdfYPPAo_11
read new file
   start    end  label
0  26.88  27.96      0
26.88 27.96 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_2.wav
MoviePy - Done.
dd
file name:bGFkdfYPPAo_12
read new file
   start     end  label
0  27.96  28.859     10
27.96 28.859 10
MoviePy - Writing audio in .\data\audio_label_clip\shi_1.wav

chunk:   0%|          | 0/28 [00:00<?, ?it/s, now=None]

...

MoviePy - Writing audio in .\data\audio_label_clip\negative_6.wav
MoviePy - Done.
dd
file name:bGFkdfYPPAo_4
read new file
   start    end  label
0    7.7  10.44      0
7.7 10.44 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_7.wav
MoviePy - Done.
dd
file name:bGFkdfYPPAo_5
read new file
   start     end  label
0  10.44  12.599      0
10.44 12.599 0


chunk:   0%|          | 0/69 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in .\data\audio_label_clip\negative_8.wav
MoviePy - Done.
dd
file name:bGFkdfYPPAo_6
read new file
    start     end  label
0  12.599  15.719      0
12.599 15.719 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_9.wav


MoviePy - Done.
dd
file name:bGFkdfYPPAo_7
read new file
    start    end  label
0  15.719  19.02      0
15.719 19.02 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_10.wav
MoviePy - Done.
dd
file name:bGFkdfYPPAo_8
read new file
   start    end  label
0  19.02  22.14      0
19.02 22.14 0
MoviePy - Writing audio in .\data\audio_label_clip\negative_11.wav


MoviePy - Done.
dd
file name:bGFkdfYPPAo_9
read new file
   start     end  label
0  22.14  25.439     10
22.14 25.439 10
MoviePy - Writing audio in .\data\audio_label_clip\shi_5.wav
MoviePy - Done.
dd
